# FIBROSIS PULMONAR - Kaggle Competition 

https://www.kaggle.com/c/osic-pulmonary-fibrosis-progression/overview

Data Description

The aim of this competition is to predict a patient’s severity of decline in lung function based on a CT scan of their lungs. Lung function is assessed based on output from a spirometer, which measures the forced vital capacity (`FVC`), i.e. the volume of air exhaled.

In the dataset, you are provided with a baseline chest CT scan and associated clinical information for a set of patients. A patient has an image acquired at time  `Week = 0`  and has numerous follow up visits over the course of approximately 1-2 years, at which time their  `FVC`  is measured.

-   In the training set, you are provided with an anonymized, baseline CT scan and the entire history of FVC measurements.
-   In the test set, you are provided with a baseline CT scan and only the initial FVC measurement.  **You are asked to predict the final three  `FVC`  measurements for each patient, as well as a confidence value in your prediction.**

Since this is real medical data, you will notice the relative timing of  `FVC`  measurements varies widely. The timing of the initial measurement relative to the CT scan and the duration to the forecasted time points may be different for each patient. This is considered part of the challenge of the competition. To avoid potential leakage in the timing of follow up visits, you are asked to predict every patient's  `FVC`  measurement for every possible week. Those weeks which are not in the final three visits are ignored in scoring.

## Files

This is a synchronous rerun code competition. The provided test set is a small representative set of files (copied from the training set) to demonstrate the format of the private test set. When you submit your notebook, Kaggle will rerun your code on the test set, which contains unseen images.

-   **train.csv**  - the training set, contains full history of clinical information
-   **test.csv**  - the test set, contains only the baseline measurement
-   **train/**  - contains the training patients' baseline CT scan in DICOM format
-   **test/**  - contains the test patients' baseline CT scan in DICOM format
-   **sample_submission.csv**  - demonstrates the submission format

## Columns

### train.csv and test.csv

-   `Patient`- a unique Id for each patient (also the name of the patient's DICOM folder)
-   `Weeks`- the relative number of weeks pre/post the baseline CT (may be negative)
-   `FVC`  - the recorded lung capacity in ml
-   `Percent`- a computed field which approximates the patient's FVC as a percent of the typical FVC for a person of similar characteristics
-   `Age`
-   `Sex`
-   `SmokingStatus`

# Fibrosis pulmonar

La fibrosis pulmonar es una afección en donde el tejido profundo de sus pulmones se va cicatrizando. Esto hace que el tejido se vuelva grueso y duro. Esto dificulta recuperar el aliento y es posible que la sangre no reciba suficiente oxígeno.

Las causas de la fibrosis pulmonar incluyen contaminantes ambientales, algunos medicamentos, enfermedades del tejido conectivo o la enfermedad pulmonar intersticial (grupo de enfermedades que causan inflamación o cicatrización en los pulmones) pero en la mayoría de los casos, no se encuentra una causa. Esto se llama fibrosis pulmonar idiopática.

![Fibrosis pulmonar - Síntomas y causas - Mayo Clinic](https://www.mayoclinic.org/-/media/kcms/gbs/patient-consumer/images/2016/08/10/14/57/mcdc7_pulmonaryfibrosis-8col.jpg)

La espirometría forzada es aquella en que, tras una inspiración máxima, se le pide al paciente que realice una espiración de todo el aire, en el menor tiempo posible. Nos permite establecer diagnósticos de la patología respiratoria. 

 - Capacidad vital forzada (FVC) (se expresa en mililitros): Volumen total que expulsa el paciente desde la inspiración máxima hasta la espiración máxima. Su valor normal es mayor del 80% del valor teórico.

[  
https://www.elsevier.es/es-revista-atencion-primaria-27-articulo-principales-parametros-funcion-pulmonar-enfermedad-13049899](https://www.elsevier.es/es-revista-atencion-primaria-27-articulo-principales-parametros-funcion-pulmonar-enfermedad-13049899)  
[https://www.kaggle.com/c/osic-pulmonary-fibrosis-progression/discussion/172022](https://www.kaggle.com/c/osic-pulmonary-fibrosis-progression/discussion/172022)  [https://erj.ersjournals.com/content/40/6/1324](https://erj.ersjournals.com/content/40/6/1324)  [https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2643211/](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2643211/)  [https://vitalograph.co.uk/resources/ers-normal-values](https://vitalograph.co.uk/resources/ers-normal-values)

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from matplotlib import style

%matplotlib inline
style.use('fivethirtyeight')

In [ ]:
dir_train_dicom = '../input/osic-pulmonary-fibrosis-progression/train'
dir_test_dicom = '../input/osic-pulmonary-fibrosis-progression/test'

dir_train_csv = '../input/osic-pulmonary-fibrosis-progression/train.csv'
dir_test_csv = '../input/osic-pulmonary-fibrosis-progression/test.csv'

dir_submission_csv = '../input/osic-pulmonary-fibrosis-progression/sample_submission.csv'

In [ ]:
train = pd.read_csv(dir_train_csv)
test = pd.read_csv(dir_test_csv)
sample_submission = pd.read_csv(dir_submission_csv)

In [ ]:
train

In [ ]:
test

In [ ]:
sample_submission

In [ ]:
train.isnull().sum()

In [ ]:
train.Patient.nunique()

In [ ]:
train.describe()

In [ ]:
sns.distplot(train.FVC)

In [ ]:
sns.distplot(train.Age, color='g')

In [ ]:
sns.distplot(train.Weeks, color='r')

## Outliers

In [ ]:
f, axes = plt.subplots(4, figsize=(15, 15))
sns.boxplot(train.FVC, ax=axes[0])
sns.boxplot(train.Age, color='g', ax=axes[1])
sns.boxplot(train.Percent, color='y', ax=axes[2])
sns.boxplot(train.Weeks, color='r', ax=axes[3])

## Relaciones existentes por sexo

In [ ]:
sns.pairplot(train, hue='Sex')

In [ ]:
sns.countplot(x=train.Sex)

In [ ]:
Porcentaje_hombres = train.Sex[train.Sex=='Male'].count()/len(train.Sex)
Porcentaje_mujeres = train.Sex[train.Sex=='Female'].count()/len(train.Sex)

print(f'Porcentaje train HOMBRES: {Porcentaje_hombres}')
print(f'Porcentaje train MUJERES: {Porcentaje_mujeres}')

In [ ]:
Porcentaje_hombres = test.Sex[train.Sex=='Male'].count()/len(test.Sex)
Porcentaje_mujeres = test.Sex[train.Sex=='Female'].count()/len(test.Sex)

print(f'Porcentaje test HOMBRES: {Porcentaje_hombres}')
print(f'Porcentaje test MUJERES: {Porcentaje_mujeres}')

## Relaciones existentes por Smoking Status

In [ ]:
sns.countplot(train.SmokingStatus)

In [ ]:
Porcentaje_exfumadores = train.SmokingStatus[train.SmokingStatus=='Ex-smoker'].count()/len(train.SmokingStatus)
Porcentaje_nuncafumaron = train.SmokingStatus[train.SmokingStatus=='Never smoked'].count()/len(train.SmokingStatus)
Porcentaje_fuman = train.SmokingStatus[train.SmokingStatus=='Currently smokes'].count()/len(train.SmokingStatus)

print(f'Porcentaje train EX-FUMADORES: {Porcentaje_exfumadores}')
print(f'Porcentaje train NUNCA FUMARON: {Porcentaje_nuncafumaron}')
print(f'Porcentaje train FUMAN: {Porcentaje_fuman}')

In [ ]:
Porcentaje_exfumadores = test.SmokingStatus[test.SmokingStatus=='Ex-smoker'].count()/len(test.SmokingStatus)
Porcentaje_nuncafumaron = test.SmokingStatus[test.SmokingStatus=='Never smoked'].count()/len(test.SmokingStatus)
Porcentaje_fuman = test.SmokingStatus[test.SmokingStatus=='Currently smokes'].count()/len(test.SmokingStatus)

print(f'Porcentaje test EX-FUMADORES: {Porcentaje_exfumadores}')
print(f'Porcentaje test NUNCA FUMARON: {Porcentaje_nuncafumaron}')
print(f'Porcentaje test FUMAN: {Porcentaje_fuman}')

In [ ]:
sns.pairplot(train, hue='SmokingStatus')

## Correlaciones 

In [ ]:
sns.heatmap(train.corr(), annot=True, fmt='.2f', cmap='YlGnBu')

## Edades mujeres

In [ ]:
train.Sex[train.Sex=='Female'].groupby(train.Age).value_counts()

In [ ]:
sns.distplot(train.Age[train.Sex=='Female'].values)

In [ ]:
train.Age[train.Sex=='Female'].describe()

## Edades hombres

In [ ]:
train.Sex[train.Sex=='Male'].groupby(train.Age).value_counts()

In [ ]:
sns.distplot(train.Age[train.Sex=='Male'].values)

In [ ]:
train.Age[train.Sex=='Male'].describe()

## Relaciones de acuerdo al FVC

In [ ]:
sns.set(rc={'figure.figsize':(15,10)})

sns.distplot(train[train.SmokingStatus == 'Ex-smoker'].FVC)
sns.distplot(train[train.SmokingStatus =='Never smoked'].FVC)
sns.distplot(train[train.SmokingStatus =='Currently smokes'].FVC)

plt.legend(['Ex-smoker','Never smoked','Currently smokes'])

In [ ]:
sns.distplot(train[train.Sex =='Female'].FVC)
sns.distplot(train[train.Sex =='Male'].FVC)

plt.legend(['Female','Male'])

In [ ]:
print(train.Weeks.max(), train.Weeks.min())
print(np.linspace(train.Weeks.max(), train.Weeks.min(), 6))

In [ ]:
sns.distplot(train[train.Weeks<=23].FVC)
sns.distplot(train[(train.Weeks>23)&(train.Weeks<=50)].FVC)
sns.distplot(train[(train.Weeks>50)&(train.Weeks<=78)].FVC)
sns.distplot(train[(train.Weeks>78)&(train.Weeks<=105)].FVC)
sns.distplot(train[(train.Weeks>105)&(train.Weeks<=133)].FVC)

plt.legend(['<23 weeks','23 y 50 weeks','50 y 78 weeks', '78 y 105 weeks', '105 y 133 weeks'])

In [ ]:
train.Weeks_cortadas = pd.cut(train["Weeks"], 8)   
grupo = train.groupby(train.Weeks_cortadas)

ax = sns.barplot(x=train.Weeks_cortadas.unique(), y= grupo.FVC.mean())
plt.xticks(rotation=45)

for rect in ax.patches:
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width()/2., 1*height,'%d' % int(height),ha='center', va='bottom')

In [ ]:
print(train.Age.max(), train.Age.min())
print(np.linspace(train.Age.max(), train.Age.min(), 4))

In [ ]:
sns.distplot(train[train.Age<=62].FVC)
sns.distplot(train[(train.Age>62)&(train.Age<=75)].FVC)
sns.distplot(train[(train.Age>75)&(train.Age<=88)].FVC)

plt.legend(['<62 age','62 y 75 age','75 y 88 age'])

In [ ]:
train.Age_cortadas = pd.cut(train["Age"], 6)   
grupo = train.groupby(train.Age_cortadas)

ax = sns.barplot(x=train.Age_cortadas.unique(), y= grupo.FVC.mean())
plt.xticks(rotation=45)

for rect in ax.patches:
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width()/2., 1*height,'%d' % int(height),ha='center', va='bottom')

In [ ]:
train.groupby(train.Patient).count().Weeks.value_counts()

Podemos ver que la mayoria de los pacientes se realizan una CT en la semana 9

In [ ]:
f, axes = plt.subplots(1,3, figsize=(20, 10))

grupo = train.groupby(train.Sex)

ax0 = sns.barplot(x=train.Sex.unique(), y= grupo.FVC.min(), ax=axes[0])
ax1 = sns.barplot(x=train.Sex.unique(), y= grupo.FVC.mean(), ax=axes[1])
ax2 = sns.barplot(x=train.Sex.unique(), y= grupo.FVC.max(), ax=axes[2])

for rect in ax0.patches:
    height = rect.get_height()
    ax0.text(rect.get_x() + rect.get_width()/2., 1*height,'%d' % int(height),ha='center', va='bottom')
    
for rect in ax1.patches:
    height = rect.get_height()
    ax1.text(rect.get_x() + rect.get_width()/2., 1*height,'%d' % int(height),ha='center', va='bottom')

for rect in ax2.patches:
    height = rect.get_height()
    ax2.text(rect.get_x() + rect.get_width()/2., 1*height,'%d' % int(height),ha='center', va='bottom')

Hay que intercambiar las variables porque el grafico de mujeres corresponde al de hombres y viceversa 

In [ ]:
ax = sns.barplot(x=train.SmokingStatus, y=train.FVC)

for rect in ax.patches:
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width()/2., 1.05*height,'%d' % int(height),ha='center', va='bottom')

[  
https://www.elsevier.es/es-revista-atencion-primaria-27-articulo-principales-parametros-funcion-pulmonar-enfermedad-13049899](https://www.elsevier.es/es-revista-atencion-primaria-27-articulo-principales-parametros-funcion-pulmonar-enfermedad-13049899)  
[https://www.kaggle.com/c/osic-pulmonary-fibrosis-progression/discussion/172022](https://www.kaggle.com/c/osic-pulmonary-fibrosis-progression/discussion/172022)  [https://erj.ersjournals.com/content/40/6/1324](https://erj.ersjournals.com/content/40/6/1324)  [https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2643211/](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2643211/)  [https://vitalograph.co.uk/resources/ers-normal-values](https://vitalograph.co.uk/resources/ers-normal-values)

A partir del ultimo link deducimos que la FVC max para hombres es de 5960 ml y 4890 para mujeres con una altura maxima de 195 cm

In [ ]:
train.Age[train.Sex=='Male'].max(), train.Age[train.Sex=='Male'].min()

In [ ]:
train.Age[train.Sex=='Female'].max(), train.Age[train.Sex=='Female'].min()

In [ ]:
train[(train.Sex=='Male')&(train.FVC>5960)]

In [ ]:
train[(train.Sex=='Female')&(train.FVC>4890)]

## Relacion de FVC vs Weeks para pacientes random

In [ ]:
Paciente_0 = train.Patient[0]
Paciente = train[train.Patient==Paciente_0]
sns.lineplot(x=Paciente.Weeks, y=Paciente.FVC), 
plt.title(Paciente_0)

In [ ]:
Paciente_10 = train.Patient[10]
Paciente = train[train.Patient==Paciente_10]
sns.lineplot(x=Paciente.Weeks, y=Paciente.FVC), 
plt.title(Paciente_10)

In [ ]:
Paciente_20 = train.Patient[20]
Paciente = train[train.Patient==Paciente_20]
sns.lineplot(x=Paciente.Weeks, y=Paciente.FVC), 
plt.title(Paciente_20)

## Utilización de libreria pydicom para trabajar con archivos dicom

DICOM (Digital Imaging and Communication On Medicine) es un estándar de transmisión de imágenes médicas y datos entre hardware de propósito médico. Las aplicaciones más comunes de este estándar son la visualización, almacenamiento, impresión y transmisión de las imágenes.
Independientemente del uso, siempre se utiliza el mismo formato, incluyendo el uso de ficheros y de red. DICOM se diferencia de otros ficheros de datos en que agrupa la información dentro de un conjunto de datos. Es decir, una radiografía de TORAX contiene el ID de paciente junto con ella, de manera que la imagen no puede ser separada por error de su información.

Los ficheros DICOM consisten en una cabecera con campos estandarizados y de forma libre, y un cuerpo con datos de imagen. 


DICOM es la abreviatura del término en inglés Digital Imaging and Communications in Medicine (Imágenes digitales y comunicaciones en medicina). Es una norma médica que describe cómo debe intercambiarse y administrarse la información relativa a imágenes médicas. Otra norma relacionada es DICOM, apartado 14, publicada por el Colegio Americano de Radiología (ACR, del inglés American College of Radiology) y la Asociación Nacional de Fabricantes Eléctricos (NEMA, del inglés National Electrical Manufacturers Association).

El apartado 14 de la norma DICOM establece unas estrictas directrices respecto a la forma en que debe realizarse la calibración de la escala de grises y las pruebas de garantía de calidad en los  [monitores utilizados para imágenes médicas](https://www.barco.com/es/glossary/healthcare/medical-display).

![](https://az877327.vo.msecnd.net/~/media/images/healthcare/medical%20glossary/dicom%20jpg.jpg?h=149&w=400&la=es&v=1)

In [ ]:
!pip install git+https://github.com/fastai/fastai2 
!pip install git+https://github.com/fastai/fastcore

In [ ]:
import pydicom
from fastai2.basics import *
from fastai2.callback.all import *
from fastai2.vision.all import *
from fastai2.medical.imaging import *

In [ ]:
train_dcm = get_dicom_files(dir_train_dicom)
train_dcm

In [ ]:
dcm_random = train_dcm[6]
dimg = dcmread(dcm_random)
print(f'El paciente random es: {dcm_random}\n') 
print(dimg)

In [ ]:
dimension = (int(dimg.Rows), int(dimg.Columns), len(dimg.PixelData))
dimension

In [ ]:
dimg.show(figsize=(8,8))

# Escala Hounsfield


La  **escala de Unidades Hounsfield**  (‘escala Hounsfield’ o ‘escala de números TC’) nombrada en honor del ingeniero y  [Premio Nobel de Fisiología o Medicina](https://es.wikipedia.org/wiki/Premios_Nobel_de_Fisiolog%C3%ADa_y_Medicina "Premios Nobel de Fisiología y Medicina")  inglés  [Sir Godfrey Newbold Hounsfield](https://es.wikipedia.org/wiki/Sir_Godfrey_Newbold_Hounsfield "Sir Godfrey Newbold Hounsfield"), es una escala cuantitativa utilizada en los estudios de  [tomografía axial computarizada](https://es.wikipedia.org/wiki/Tomograf%C3%ADa_axial_computarizada "Tomografía axial computarizada")  para describir los diferentes niveles de radiodensidad de los tejidos humanos.


## Definición

La escala de Unidades Hounsfield (símbolo  **HU**  del inglés ‘_Hounsfield Units_’) es el resultado de la transformación de la escala de coeficientes de atenuación lineal de rayos X en una nueva escala en la cual el valor de atenuación del agua destilada en Condiciones Normales de Presión y Temperatura ([CNPT](https://es.wikipedia.org/wiki/Condiciones_normales_de_presi%C3%B3n_y_temperatura "Condiciones normales de presión y temperatura")) se define como 0 unidades de Hounsfield (HU), mientras que la radiodensidad del aire en Condiciones Normales de Presión y Temperatura (CNPT) se define como -1000 HU, extendiéndose más allá de las 1000 HU asignadas al nivel de absorción del hueso compacto.

## Fórmula

El coeficiente de atenuación de los rayos X expresado en Unidades Hounsfield para los diferentes tejidos vivos u otros materiales sometidos a un examen tomográfico de rayos X viene dado por la siguiente fórmula:

![{\displaystyle HU=1000\times {\frac {\mu _{t}-\mu _{agua}}{\mu _{agua}}}}](https://wikimedia.org/api/rest_v1/media/math/render/svg/499d76d1b25e18e4c37001a66c231a186303a567)

![{\displaystyle \mu _{agua}}](https://wikimedia.org/api/rest_v1/media/math/render/svg/144765c4e4972847ee8e881d72cf9315266b575e)  es el coeficiente de atenuación lineal del agua destilada y  ![{\displaystyle \mu _{t}}](https://wikimedia.org/api/rest_v1/media/math/render/svg/d8247e7beb0b3fa21c4fe35defef3a3cf93df67d)  el coeficiente de atenuación lineal del tejido o material de interés.

## Rango absoluto y ventanas

Aunque teóricamente la escala Hounsfield es una escala abierta, lo cierto es que en el ámbito del diagnóstico por imagen se ha establecido un rango de valores útiles que abarca desde -1024 HU hasta 3071 HU; estos 4096 niveles de gris pueden representarse mediante un número de 12 bits (212= 4096).

Debido a que la percepción humana solo permite distinguir a simple vista un máximo de 32 niveles diferenciados de gris (25), los valores absolutos registrados por el sistema de rayos X suelen explorarse mediante la generación matemática de ventanas. Una ventana es el resultado de transportar linealmente los valores de un rango de densidades de la escala Hounsfield (12 bits y 4096 niveles) para ser representados en una escala de grises de 32 niveles (5 bits).

La exploración mediante ventanas permite diferenciar con gran claridad estructuras que poseen una pequeña diferencia expresada en unidades Hounsfield (HU).

## Valor de Unidades Hounsfield (HU) para los tejidos y sustancias del cuerpo humano
|Sustancia|HU  |
|--|--|
| Aire |-1000  |
| Pulmon|-500  |
| Grasa|-100 a 55  |
| Agua| 0  |
| Liquido Cerebroespinal|15|
| Riñon|30|
| Sangre|30 a 45|
| Musculo|10 a 40|
| Sustancia Gris|37 a 45|
| Sustancia Blanca|20 a 30|
| Higado|40 a 60|



https://stackoverflow.com/questions/8756096/window-width-and-center-calculation-of-dicom-image/8765366#8765366

In [ ]:
pix = dimg.pixels.flatten()
sns.distplot(pix, kde=False)

In [ ]:
dimg.show(max_px=-500, min_px=-1200, figsize=(8,8))

In [ ]:
from skimage.measure import label,regionprops
from skimage.segmentation import clear_border

In [ ]:
img = clear_border(dimg)
plt.figure(figsize=(10,10))
plt.imshow(img, cmap=plt.cm.bone)
plt.axis('off')